# Quantum Connection Setup

In [ ]:
from pathlib import Path
import sys, importlib

sys.path.append(str(Path.cwd().resolve() / "src"))

from scenarios import *

def reload_modules():
    """
    This function reloads modules so that changes in the files are
    reflected in the notebook without requiring a kernel restart.
    """
    global longRange, direct, comparison, distillation
    if 'plot_longRange' in sys.modules:
        longRange = importlib.reload(sys.modules['plot_longRange'])
    else:
        import plot_longRange as longRange

    if 'plot_directComm' in sys.modules:
        direct = importlib.reload(sys.modules['plot_directComm'])
    else:
        import plot_directComm as direct
    if 'plot_comparison' in sys.modules:
        comparison = importlib.reload(sys.modules['plot_comparison'])
    else:
        import plot_comparison as comparison
    if 'plot_distillSwap_vs_swapDistill' in sys.modules:
        distillation = importlib.reload(sys.modules['plot_distillSwap_vs_swapDistill'])
    else:
        import plot_distillSwap_vs_swapDistill as distillation

# Analysis Functions

In [ ]:
import re
def get_img_path(label:str):
    # resolve output directory relative to this file (avoid changing cwd)
    base = Path.cwd().resolve()  # repo root
    subfolder = re.sub(r'\s+', '_', label.splitlines()[0])
    outdir = base / "img" / subfolder
    label = label.split(maxsplit=2)[0] + "_" + label.splitlines()[1]
    outdir.mkdir(parents=True, exist_ok=True)
    label = re.sub(r'\s+', '_', str(label)).strip()          # collapse whitespace/newlines
    label = re.sub(r'[^A-Za-z0-9._-]', '_', label)           # keep a safe subset of chars
    label = re.sub(r'[()]', '', label)
    label = label[:190] + ".png"
    imgpath = outdir / label
    return imgpath 

# Run Simulation

In [ ]:
reload_modules()
all_res_long = longRange.run_longrange_sims(param_sets)

In [ ]:
import copy
param_direct = copy.deepcopy(param_sets)
for p in param_direct:
    p["distances"] = [2*d for d in p["distances"]]
autofill_params(param_direct)

reload_modules()
all_res_direct = direct.run_sims(param_direct, 1e-3)

In [ ]:
reload_modules()
direct.plot_sims(all_res_direct)
longRange.plot_longrange(all_res_long)

In [ ]:
reload_modules()
comparison.print_comparison_table(all_res_long, all_res_direct)
comparison.plot_comparison(all_res_long, all_res_direct)

In [ ]:
reload_modules()
all_res_distill_then = distillation.run_distill_then_swap_sims(param_sets)
all_res_swap_then = distillation.run_swap_then_distill_sims(param_sets)

In [ ]:
reload_modules()
distillation.plot_distill_then_vs_swap_then(all_res_distill_then, all_res_swap_then)